In [1]:
from vllm import LLM, SamplingParams
import json
from tqdm import tqdm
import time

INFO 03-31 16:08:24 [__init__.py:239] Automatically detected platform cuda.


In [2]:
llm = LLM(
    # model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    model="google/gemma-3-4b-it",
    dtype="bfloat16",
    tensor_parallel_size=1,
    gpu_memory_utilization=0.9,
    max_model_len=1024
)

# Configure generation parameters
sampling_params = SamplingParams(
    temperature=0.6,
    top_p=0.95,
    max_tokens=256  # Adjust based on your needs
)

# # Configure generation parameters
# sampling_params = SamplingParams(
#     temperature=1.0,
#     top_p=0.998,
#     max_tokens=256  # Adjust based on your needs
# )



INFO 03-31 16:08:36 [config.py:585] This model supports multiple tasks: {'score', 'embed', 'reward', 'generate', 'classify'}. Defaulting to 'generate'.
INFO 03-31 16:08:36 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 03-31 16:08:38 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='google/gemma-3-4b-it', speculative_config=None, tokenizer='google/gemma-3-4b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=N

2025-03-31 16:08:38,435 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend


WARNING 03-31 16:08:38 [utils.py:2321] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x72dd5fc6d910>
INFO 03-31 16:08:38 [parallel_state.py:954] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0
INFO 03-31 16:08:38 [cuda.py:220] Using Flash Attention backend on V1 engine.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


INFO 03-31 16:08:43 [gpu_model_runner.py:1174] Starting to load model google/gemma-3-4b-it...
INFO 03-31 16:08:43 [config.py:3243] cudagraph sizes specified by model runner [1, 2, 4, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200, 208, 216, 224, 232, 240, 248, 256, 264, 272, 280, 288, 296, 304, 312, 320, 328, 336, 344, 352, 360, 368, 376, 384, 392, 400, 408, 416, 424, 432, 440, 448, 456, 464, 472, 480, 488, 496, 504, 512] is overridden by config [512, 384, 256, 128, 4, 2, 1, 392, 264, 136, 8, 400, 272, 144, 16, 408, 280, 152, 24, 416, 288, 160, 32, 424, 296, 168, 40, 432, 304, 176, 48, 440, 312, 184, 56, 448, 320, 192, 64, 456, 328, 200, 72, 464, 336, 208, 80, 472, 344, 216, 88, 120, 480, 352, 248, 224, 96, 488, 504, 360, 232, 104, 496, 368, 240, 112, 376]
INFO 03-31 16:08:43 [topk_topp_sampler.py:53] Using FlashInfer for top-p & top-k sampling.
INFO 03-31 16:08:43 [weight_utils.py:265] Using model weights format ['*.safet

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 03-31 16:08:45 [loader.py:447] Loading weights took 1.10 seconds
INFO 03-31 16:08:45 [gpu_model_runner.py:1186] Model loading took 8.5833 GB and 1.587611 seconds
INFO 03-31 16:08:45 [gpu_model_runner.py:1456] Encoder cache will be initialized with a budget of 8192 tokens, and profiled with 32 image items of the maximum feature size.
INFO 03-31 16:08:56 [backends.py:415] Using cache directory: /home/asjchoi/.cache/vllm/torch_compile_cache/b2cfc5c0b6/rank_0_0 for vLLM's torch.compile
INFO 03-31 16:08:56 [backends.py:425] Dynamo bytecode transform time: 7.37 s
INFO 03-31 16:08:57 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 03-31 16:09:03 [monitor.py:33] torch.compile takes 7.37 s in total


2025-03-31 16:09:03,875 - INFO - flashinfer.jit: Loading JIT ops: sampling
/home/asjchoi/Desktop/Code/UVD/llm-venv/lib/python3.12/site-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/home/asjchoi/Desktop/Code/UVD/llm-venv/lib/python3.12/site-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
2025-03-31 16:09:03,886 - INFO - flashinfer.jit: Finished loading JIT ops: sampling


INFO 03-31 16:09:04 [kv_cache_utils.py:566] GPU KV cache size: 26,608 tokens
INFO 03-31 16:09:04 [kv_cache_utils.py:569] Maximum concurrency for 1,024 tokens per request: 25.98x
INFO 03-31 16:09:19 [gpu_model_runner.py:1534] Graph capturing finished in 14 secs, took 0.55 GiB
INFO 03-31 16:09:20 [core.py:151] init engine (profile, create kv cache, warmup model) took 35.27 seconds


In [4]:
def wrap_prompt(prompt: str) -> str:
    return f'<｜User｜> {prompt} <｜Assistant｜>'

with open("val_language.json", "r") as f:
    prompts = json.load(f)


example1 = "put green squash into pot or pan"  #2
example2 = 'Pick the carrot from left side and place it on the orange cloth'  # 3
example3 = 'open the drawer'
example4 = 'uncover the pot and put the lid on the left side of the cloth'

sentences = list(prompts.values())[2210:2220]

for lang, num_sg in sentences:
    system_prompt = (f"I need to decompose a language command into 2 smaller language commands. "
                     f"This language command consists of grasping an object and placing it somewhere. "
                     f"Here are two examples. "
                     f"Example 1: Decompose '{example1}' into 2 smaller language commands. Output format:\n"
                     f"- grasp the green squash.\n"
                     f"- place the green squash into the pot or pan.\n"
                     f"Example 2: Decompose '{example2}' into 2 smaller language commands. Output format:\n"
                     f"- grab the carrot from the left side.\n"
                     f"- place the carrot on the orange cloth.\n"
                     f"Example 3: Decompose '{example3}' into 2 smaller language commands. Output format:\n"
                     f"- grasp the drawer handling.\n"
                     f"- pull the drawer open.\n"
                     f"Example 4: Decompose '{example4}' into 2 smaller language commands. Output format:\n"
                     f"- grab the pot lid.\n"
                     f"- put the pot lid on the left side of the cloth.\n"
                     f"Now decompose this language command: '{lang}' into exactly 2 smaller language commands. "
                     f"For each commands verb, here are the last 5 you used. Avoid using the same ones. "
                     f"Already used
                     f"IMPORTANT: You must ONLY output the 2 bullet points with no additional text, explanations, or thinking process. "
                     f"Start each command with a dash and a space.")

    # Approximate
    num_tokens = len(system_prompt.split())
    assert num_tokens < 1024

    system_prompt = wrap_prompt(system_prompt)
    #
    outputs = llm.generate([system_prompt], sampling_params)

    # Extract and store the generated text
    generated_text = outputs[0].outputs[0].text
    print(lang)
    print(generated_text)

# results.append({
#     "prompt_id": i,
#     "prompt": prompt,
#     "generated_text": generated_text
# })





Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.68it/s, est. speed input: 774.69 toks/s, output: 48.25 toks/s]


take the lid and covered it to the silver pot

- grasp the lid.
- cover the lid to the silver pot.



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.80it/s, est. speed input: 805.06 toks/s, output: 56.10 toks/s]


put the purple towel in the washing machine

- grasp the purple towel.
- place the purple towel in the washing machine.



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.78it/s, est. speed input: 787.14 toks/s, output: 47.28 toks/s]


...

put the spoon on the table

- grasp the spoon.
- place the spoon on the table.



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.76it/s, est. speed input: 1080.51 toks/s, output: 56.47 toks/s]


take the silver lead and close the pot

- grasp the silver lead.
- close the pot.



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.60it/s, est. speed input: 752.19 toks/s, output: 57.26 toks/s]

Move the yellow cloth to the centre of the table

- grasp the yellow cloth.
- place the yellow cloth in the centre of the table.

